In [ ]:
import time
import sys
import os
import numpy as np

%load_ext autoreload
%autoreload 2

import xfmkit.utils as utils
import xfmkit.argops as argops
import xfmkit.clustering as clustering
import xfmkit.visualisations as vis
import xfmkit.processops as processops
import xfmkit.structures as structures
import xfmkit.entry_processed as entry_processed

#add the parent folder to sys path so we can import from the notebook subfolder
sys.path.insert(0,'..')

#args = ' -d /home/lachlan/CODEBASE/xfmkit/data/processed_maps/nf_demo_short -n 2'
args = ["-d", "/home/lachlan/CODEBASE/xfmkit/data/processed_maps/carlos_full", "-n", "2", "-fc" ]

pxs, embedding, categories, classavg, palette = entry_processed.read_processed(args)





In [ ]:
import xfmkit.tabular as tabular

df = tabular.get_df(classavg, pxs.labels)
major_list = tabular.get_major_list(df)
major_strings = tabular.nestlist_as_str(major_list)


len(major_list), len(classavg), utils.count_categories(categories)[0]

In [ ]:
fig_embed = vis.seaborn_embedplot(embedding, categories, palette=palette, labels=major_strings)

In [ ]:
#vis.tricolour_enames("Fe", "Ti", "AgL", pxs.data.d, pxs.dimensions, pxs.labels)

vis.show_map(pxs.data.d, pxs.dimensions, pxs.labels, "Ti")

In [ ]:
ca = clustering.get_classavg(pxs.data.d, categories, '.', force=True, overwrite=False)



In [ ]:
ca[30,22]